In [75]:
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
import time
import re
# Use package tmdbsimple to extract data
import tmdbsimple as tmdb
# use "!pip install tmdbsimple" to install
tmdb.API_KEY = '302f3815bea132a8bfe0d7301c9065dd'
import random
import sys
import time

In [76]:
df_tmdb = pd.read_csv('TMDb_data_25K.csv',index_col=0)
df_tmdb.head()

,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,[u'United States of America'],4300000,It's Ted the Bellhop's first night on the job....,False,5,"[u'Crime', u'Comedy']",Four Rooms,Twelve outrageous guests. Four scandalous requ...,402,...,tt0113101,False,/3EqYpbGCE9S5GddU2K4cYzP5UmI.jpg,"[u'Miramax Films', u'A Band Apart']",12/25/1995,1.048212,Four Rooms,4000000,6.4,98.0
1,NaN,[u'Germany'],0,NaN,False,9,[u'Drama'],Sunday in August,NaN,1,...,tt0425473,False,NaN,[],9/2/2004,0.003213,Sonntag im August,0,1.0,15.0
2,/tvSlBzAdRE29bZe5yYWrJ2ds137.jpg,[u'United States of America'],775398007,Princess Leia is captured and held hostage by ...,False,11,"[u'Adventure', u'Action', u'Science Fiction']",Star Wars,"A long time ago in a galaxy far, far away...",5333,...,tt0076759,False,/4iJfYYoQzZcONB9hNzg0J0wWyPH.jpg,"[u'Lucasfilm', u'Twentieth Century Fox Film Co...",5/25/1977,10.412617,Star Wars,11000000,8.0,121.0
3,/zjqInUwldOBa0q07fOyohYCWxWX.jpg,[u'United States of America'],940335536,A tale which follows the comedic and eventful ...,False,12,"[u'Animation', u'Family']",Finding Nemo,"There are 3.7 trillion fish in the ocean, they...",4426,...,tt0266543,False,/n2vIGWw4ezslXjlP0VNxkp9wqwU.jpg,"[u'Walt Disney Pictures', u'Pixar Animation St...",5/30/2003,4.987594,Finding Nemo,94000000,7.5,100.0
4,/8fzjzQhLXl1afshhsE5Y3MGuco4.jpg,"[u'Germany', u'United Kingdom']",0,Ad??le and her daughter Sarah are traveling on...,False,17,"[u'Horror', u'Thriller', u'Mystery']",The Dark,One of the living for one of the dead.,61,...,tt0411267,False,/pkCC3myn5QuoLzplTlvZLB0w2AU.jpg,"[u'Constantin Film', u'Impact Pictures', u'Isl...",1/26/2006,0.347210,The Dark,0,5.5,87.0


In [77]:
df_imdb = pd.read_csv('IMDb_25K_TopHalf.csv')
df_imdb.head()

,Unnamed: 0,title,genres,director,distributors,year,rating,votes,runtimes,language codes,...,original music,canonical title,editorial department,canonical title.1,long imdb title,long imdb canonical title,smart canonical title,smart long imdb canonical title,full-size cover url,imdb_ids
0,0,Four Rooms,[u'Comedy'],"[<Person id:0025978[http] name:_Anders, Alliso...",[<Company id:0051881[http] name:_Concorde Film...,1995.0,6.7,81543.0,"[u'98', u'110::(workprint version)']",[u'en'],...,"[<Person id:0249375[http] name:_Edison, Combus...",Four Rooms,"[<Person id:0020317[http] name:_Allen, Bob_>, ...",Four Rooms,Four Rooms (1995),Four Rooms (1995),Four Rooms,Four Rooms (1995),https://images-na.ssl-images-amazon.com/images...,113101
1,1,"Sonntag, im August",[u'Short'],"[<Person id:1262754[http] name:_Meyer, Marc_>]",NaN,2004.0,6.5,11.0,[u'15'],[u'de'],...,"[<Person id:1909419[http] name:_Biegai, Christ...","Sonntag, im August",NaN,"Sonntag, im August","Sonntag, im August (2004)","Sonntag, im August (2004)","Sonntag, im August","Sonntag, im August (2004)",https://images-na.ssl-images-amazon.com/images...,425473
2,2,Star Wars,"[u'Action', u'Adventure', u'Fantasy', u'Sci-Fi']","[<Person id:0000184[http] name:_Lucas, George_>]",[<Company id:0000756[http] name:_Twentieth Cen...,1977.0,8.7,969951.0,"[u'121', u'125::(special edition)']",[u'en'],...,"[<Person id:0002354[http] name:_Williams, John_>]",Star Wars,"[<Person id:0091250[http] name:_Boekelheide, T...",Star Wars,Star Wars (1977),Star Wars (1977),Star Wars,Star Wars (1977),https://images-na.ssl-images-amazon.com/images...,76759
3,3,Finding Nemo,"[u'Animation', u'Adventure', u'Comedy', u'Fami...","[<Person id:0004056[http] name:_Stanton, Andre...","[<Company id:0358618[http] name:_BDG filmai_>,...",2003.0,8.1,743705.0,[u'100'],[u'en'],...,"[<Person id:0002353[http] name:_Newman, Thomas_>]",Finding Nemo,"[<Person id:0023224[http] name:_y Alvarez, Lui...",Finding Nemo,Finding Nemo (2003),Finding Nemo (2003),Finding Nemo,Finding Nemo (2003),https://images-na.ssl-images-amazon.com/images...,266543
4,4,The Dark,"[u'Horror', u'Mystery', u'Thriller']","[<Person id:0269502[http] name:_Fawcett, John_>]",[<Company id:0002257[http] name:_Constantin Fi...,2005.0,5.4,9415.0,"[u'93', u'Philippines:92::(cut)']","[u'en', u'cy']",...,"[<Person id:0125220[http] name:_Butt, Edmund_>]","Dark, The","[<Person id:1262315[http] name:_Alexander, Mar...","Dark, The",The Dark (2005),"Dark, The (2005)","Dark, The","Dark, The (2005)",https://images-na.ssl-images-amazon.com/images...,411267


In [78]:
# create a dataframe to store information of top 500 popular movies
columns = ['Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy', 'Family', 'Romance', 'Horror', 'Reality-TV', 'Western', 'Documentary', 'Biography', 'News', 'Film-Noir', 'Drama', 'Animation', 'Sci-Fi', 'Thriller', 'Short', 'Mystery', 'Sport', 'War', 'History', 'Game-Show', 'Music', 'Musical', 'Talk-Show', 'Foreign', 'TV movie']
index = range(len(df_tmdb)) 
df_merge_y = pd.DataFrame(index = index, columns=columns)
df_merge_y = df_merge_y.fillna(0)

In [79]:
# function genre_assign
# input: genres for one movie, string type
# output: a string list, which are corresponding columns names for the genres of input movie 
def genre_assign(genre_temp):
    columns_list = ['Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy', 'Family', 'Romance', 'Horror', 'Reality-TV', 'Western', 'Documentary', 'Biography', 'News', 'Film-Noir', 'Drama', 'Animation', 'Sci-Fi', 'Thriller', 'Short', 'Mystery', 'Sport', 'War', 'History', 'Game-Show', 'Music', 'Musical', 'Talk-Show', 'Foreign', 'TV movie']
    genre_list = []
    for i in columns_list:
        if i in genre_temp:
            genre_list += [i]
    if "Sci-Fi" not in genre_list:
        if "Science Fiction" in genre_temp:
            genre_list += ['Sci-Fi']
    return genre_list

In [80]:
for i in range(len(df_imdb)):
    if i not in [5551, 12767, 15224, 24829]: # these movies have trouble finding TMDB info, should remove later
        genre_temp = str(df_tmdb.loc[i, "genres"]) + str(df_imdb.loc[i, "genres"])
        column_list_temp = genre_assign(genre_temp)
        for column in column_list_temp:
            df_merge_y.loc[i, column] = 1
            

In [86]:
# write the dataframe out
df_merge_y.to_csv('genre_multi_25K.csv')

---------------